# Install requirements

In [ ]:
!pip3 install openmim
!mim install mmengine

In [ ]:
!mim install "mmcv>=2.0.0"

In [ ]:
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -e .

In [ ]:
!sed -i -e 's/2.2.0/2.2.1/g' ./mmdet/__init__.py

In [ ]:
import mmdet
print(mmdet.__version__)

# Dataset preparation

In [ ]:
!gdown 1JGYVpDE1jb3sIMJd2Cxw0-p_FoYMDs-r
!unzip -q augmented_data_v1.zip

In [ ]:
!pip install -q globox
from globox import AnnotationSet
yolo_train_set = AnnotationSet.from_yolo_v5(
    folder="./augmented_data_v1/train/labels/",
    image_folder="./augmented_data_v1/train/images/"
)
yolo_val_set = AnnotationSet.from_yolo_v5(
    folder="./augmented_data_v1/valid/labels/",
    image_folder="./augmented_data_v1/valid/images/"
)
yolo_train_set.save_coco("./augmented_data_v1/train.json", auto_ids=True)
yolo_val_set.save_coco("./augmented_data_v1/valid.json", auto_ids=True)

# Deformable DETR

In [ ]:
!mim download mmdet --config deformable-detr_r50_16xb2-50e_coco --dest .

processing deformable-detr_r50_16xb2-50e_coco...
downloading ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.7/231.7 MiB 21.4 MB/s eta 0:00:0000:0100:01
Successfully downloaded deformable-detr_r50_16xb2-50e_coco_20221029_210934-6bc7d21b.pth to /teamspace/studios/this_studio/mmdetection
Successfully dumped deformable-detr_r50_16xb2-50e_coco.py to /teamspace/studios/this_studio/mmdetection


In [ ]:
%%writefile ./configs/deformable_detr/deformable-detr_r50_16xb2-50e_sino_nom.py
# Inherit and overwrite part of the config based on this config
_base_ = './deformable-detr_r50_16xb2-50e_coco.py'

data_root = 'augmented_data_v1/' # dataset root

train_batch_size_per_gpu = 1
train_num_workers = 1

max_epochs = 20
stage2_num_epochs = 1
base_lr = 0.00008


metainfo = {
    'classes': ('0', ),
    'palette': [
        (220, 20, 60),
    ]
}

train_dataloader = dict(
    batch_size=train_batch_size_per_gpu,
    num_workers=train_num_workers,
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        data_prefix=dict(img='train/images/'),
        ann_file='train.json'))

val_dataloader = dict(
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        data_prefix=dict(img='valid/images/'),
        ann_file='valid.json'))

test_dataloader = val_dataloader

val_evaluator = dict(ann_file=data_root + 'valid.json')

test_evaluator = val_evaluator

model = dict(num_queries=300, bbox_head=dict(num_classes=1), test_cfg=dict(max_per_img=300))

# learning rate
param_scheduler = [
    dict(
        type='LinearLR',
        start_factor=1.0e-5,
        by_epoch=False,
        begin=0,
        end=10),
    dict(
        # use cosine lr from 10 to 20 epoch
        type='CosineAnnealingLR',
        eta_min=base_lr * 0.05,
        begin=max_epochs // 3,
        end=max_epochs,
        T_max=max_epochs // 3,
        by_epoch=True,
        convert_to_iter_based=True),
]

train_pipeline_stage2 = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='RandomResize',
        scale=(640, 640),
        ratio_range=(0.1, 2.0),
        keep_ratio=True),
    dict(type='RandomCrop', crop_size=(640, 640)),
    dict(type='YOLOXHSVRandomAug'),
    dict(type='RandomFlip', prob=0.5),
    dict(type='Pad', size=(640, 640), pad_val=dict(img=(114, 114, 114))),
    dict(type='PackDetInputs')
]

# optimizer
optim_wrapper = dict(
    _delete_=True,
    type='OptimWrapper',
    optimizer=dict(type='AdamW', lr=base_lr, weight_decay=0.05),
    paramwise_cfg=dict(
        norm_decay_mult=0, bias_decay_mult=0, bypass_duplicate=True))

default_hooks = dict(
    checkpoint=dict(
        interval=5,
        max_keep_ckpts=2,  # only keep latest 2 checkpoints
        save_best='auto'
    ),
    logger=dict(type='LoggerHook', interval=20))

custom_hooks = [
    dict(
        type='PipelineSwitchHook',
        switch_epoch=max_epochs - stage2_num_epochs,
        switch_pipeline=train_pipeline_stage2)
]

# load COCO pre-trained weight
load_from = './deformable-detr_r50_16xb2-50e_coco_20221029_210934-6bc7d21b.pth'

train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=max_epochs, val_interval=1)
visualizer = dict(vis_backends=[dict(type='LocalVisBackend'),dict(type='TensorboardVisBackend')])

Overwriting ./configs/deformable_detr/deformable-detr_r50_16xb2-50e_sino_nom.py


In [ ]:
!python tools/train.py configs/deformable_detr/deformable-detr_r50_16xb2-50e_sino_nom.py

05/22 15:46:27 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.10 (main, Mar 21 2023, 18:45:11) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1620886897
    GPU 0: NVIDIA L4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 12.1, V12.1.105
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 2.2.1+cu121
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.2 (Git Hash 2dc95a2ad0841e29db8b22fbccaf3e5da7992b01)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arc